In [961]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
import joblib
import tensorflow as tf

# Codigo 2, esta ves con pesos

In [962]:
df = pd.read_csv('C:/Users/FRANK/Desktop/Proyectos Linkedin/Python Proyectos/Redes Neuronales/Dia 24/df3.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Churn_pred
0,CUST0000,Male,0,No,Yes,23,No,No phone service,No,No internet service,...,No internet service,No internet service,No internet service,Month-to-month,Yes,Bank transfer,49.85,1146.55,0,1
1,CUST0001,Female,0,Yes,No,43,No,No phone service,DSL,Yes,...,No,Yes,No,Month-to-month,No,Mailed check,100.70,4330.10,1,1
2,CUST0002,Male,1,No,No,51,Yes,No,DSL,No,...,Yes,No,No,One year,No,Electronic check,97.33,4963.83,1,0
3,CUST0003,Male,1,No,No,72,Yes,Yes,DSL,Yes,...,No,No,No,Month-to-month,No,Credit card,101.38,7299.36,0,1
4,CUST0004,Male,1,No,No,25,Yes,Yes,DSL,No,...,Yes,No,Yes,Month-to-month,No,Electronic check,52.22,1305.50,1,1


In [963]:
df2 = df.copy()
print('Número de observaciones (filas) = {}'.format(df2.shape[0]))
print('Total de variables disponibles (columnas) = {} '.format(df2.columns.shape[0]))
print('Columnas que cuentan con un valor nulo -')
print(df2.isnull().sum())

Número de observaciones (filas) = 5880
Total de variables disponibles (columnas) = 22 
Columnas que cuentan con un valor nulo -
customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
Churn_pred          0
dtype: int64


In [964]:
print("Valores únicos de 'Churn' después del mapeo:")
print(df2['Churn'].unique())

Valores únicos de 'Churn' después del mapeo:
[0 1]


In [965]:
# Remplazar por 0 y 1

binario = {'gender': {'Male':1,'Female':0},
          'Partner': {'Yes': 1,'No':0},
       'Dependents': {'No':0,'Yes':1},
     'PhoneService': {'Yes':1,'No':0},
 'PaperlessBilling': {'Yes':1,'No':0},}

for column, mapping in binario.items():
  df2[column] = df2[column].map(mapping)

In [966]:
# Definir características numéricas y categóricas

numeric_features = ['tenure','MonthlyCharges','TotalCharges']
categorical_features = ['MultipleLines','InternetService','OnlineBackup','OnlineSecurity','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaymentMethod']

In [967]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df2[numeric_features] = scaler.fit_transform(df2[numeric_features]) # type: ignore

In [968]:
# One-Hot Encoding para características categóricas
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_cats = enc.fit_transform(df2[categorical_features])

In [969]:
# Crear DataFrame para las características codificadas
encoded_cats_df = pd.DataFrame(encoded_cats, columns=enc.get_feature_names_out(categorical_features))
df2 = pd.concat([df2, encoded_cats_df], axis=1)

In [970]:
df2 = df2.drop(columns=['customerID','MultipleLines','OnlineBackup','InternetService','OnlineSecurity',
                        'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaymentMethod','Churn_pred'])

In [971]:
df2.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,...,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer,PaymentMethod_Credit card,PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,1,0,0,1,-0.648040,0,1,-0.705078,-0.743650,0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,0,1,0,0.308537,0,0,1.060414,0.923257,1,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1,1,0,0,0.691167,1,0,0.943409,1.255078,1,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1,1,0,0,1.695572,1,0,1.084023,2.477961,0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1,1,0,0,-0.552382,1,0,-0.622793,-0.660423,1,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


# Predicción con pesos

In [972]:
# Definir X e y
X = df2.drop(columns=['Churn'])
y = df2['Churn']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [973]:
# Reindexar y_train para que coincida con los índices de X_train si es necesario
y_train = y_train.reset_index(drop=True)
X_train = X_train.reset_index(drop=True)

In [974]:
# Calcular el número de muestras de cada clase
class_counts = y_train.value_counts() # type: ignore
total_samples = len(y_train) # type: ignore

# Calcular los pesos de cada clase
weights = { 
    0: total_samples / (2 * class_counts[0]),
    1: total_samples / (2 * class_counts[1])
}

print("Pesos de clase:", weights)

Pesos de clase: {0: 0.9808173477898249, 1: 1.0199479618386817}


In [975]:
oculta  = tf.keras.layers.Dense(units=128,activation='relu',input_shape=[40])
dropout = tf.keras.layers.Dropout(0.3)
oculta1 = tf.keras.layers.Dense(units=64,activation='relu')
dropout1 = tf.keras.layers.Dropout(0.3)
oculta2 = tf.keras.layers.Dense(units=32,activation='relu')
dropout2 = tf.keras.layers.Dropout(0.3)
oculta3 = tf.keras.layers.Dense(units=16,activation='relu')
dropout3 = tf.keras.layers.Dropout(0.3)
oculta4 = tf.keras.layers.Dense(units=8,activation='relu')
dropout4 = tf.keras.layers.Dropout(0.3)
salida = tf.keras.layers.Dense(units=1, activation='sigmoid')
modelo = tf.keras.Sequential([oculta,dropout,oculta1,dropout1,oculta2,dropout2,oculta3,oculta4,dropout4,dropout3,salida])

In [976]:
modelo.compile(optimizer=tf.keras.optimizers.Adam(0.0001),loss='binary_crossentropy',metrics=['accuracy'])

In [977]:
# Implementar el early_stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [978]:
# Entrenar el modelo
print('Comenzando entrenamiento')
historial = modelo.fit(X_train, y_train, epochs=1000, batch_size=64, verbose=False, validation_split=0.2, callbacks=[early_stopping], class_weight=weights) # type: ignore
print('Modelo entrenado')

Comenzando entrenamiento
Modelo entrenado


In [979]:
plt.xlabel('# Epocas')
plt.ylabel('Magnitud de perdida')
plt.plot(historial.history['loss'])

In [980]:
# Realizar la predicción
print('Vamos a hacer una predicción')
nuevo_dato = np.array([[1, 3, 10, 70, 10, 10, 15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])  # Ejemplo de nuevo dato con las 7 características numéricas
nuevo_dato[:, :3] = scaler.transform(nuevo_dato[:, :3])  # Escalar las características numéricas
resultado = modelo.predict(nuevo_dato)

print(f'El resultado es: {resultado[0][0]}')

Vamos a hacer una predicción
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
El resultado es: 0.13775262236595154


In [981]:
# Convertir la probabilidad a una predicción binaria
prediccion_binaria = (resultado > 0.5).astype(int)

# Imprimir el resultado y la predicción
print(f'El resultado es: {prediccion_binaria[0][0]}')
if prediccion_binaria[0][0] == 0:
    print(f'El resultado es: {prediccion_binaria[0][0]}, predicción: 0, Abandono')
else:
    print(f'El resultado es: {prediccion_binaria[0][0]}, predicción: 1, No abandono')

El resultado es: 0
El resultado es: 0, predicción: 0, Abandono


In [982]:
# Realizar predicciones usando el modelo entrenado
predicciones = modelo.predict(X)

predicciones_binarias = (predicciones > 0.5).astype(int).flatten()

df2['Churn_pred'] = predicciones_binarias

# Verificar las primeras filas del DataFrame con las predicciones
print(df2.head())

184/184 ━━━━━━━━━━━━━━━━━━━━ 0s 915us/step
   gender  SeniorCitizen  Partner  Dependents    tenure  PhoneService  \
0       1              0        0           1 -0.648040             0   
1       0              0        1           0  0.308537             0   
2       1              1        0           0  0.691167             1   
3       1              1        0           0  1.695572             1   
4       1              1        0           0 -0.552382             1   

   PaperlessBilling  MonthlyCharges  TotalCharges  Churn  ...  \
0                 1       -0.705078     -0.743650      0  ...   
1                 0        1.060414      0.923257      1  ...   
2                 0        0.943409      1.255078      1  ...   
3                 0        1.084023      2.477961      0  ...   
4                 0       -0.622793     -0.660423      1  ...   

   StreamingMovies_No internet service  StreamingMovies_Yes  \
0                                  1.0                  0.0   
1

In [983]:
df3 = df.copy()
df3['Churn_pred_1'] = predicciones_binarias
df3.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,Churn_pred,Churn_pred_1
0,CUST0000,Male,0,No,Yes,23,No,No phone service,No,No internet service,...,No internet service,No internet service,Month-to-month,Yes,Bank transfer,49.85,1146.55,0,1,0
1,CUST0001,Female,0,Yes,No,43,No,No phone service,DSL,Yes,...,Yes,No,Month-to-month,No,Mailed check,100.70,4330.10,1,1,1
2,CUST0002,Male,1,No,No,51,Yes,No,DSL,No,...,No,No,One year,No,Electronic check,97.33,4963.83,1,0,1
3,CUST0003,Male,1,No,No,72,Yes,Yes,DSL,Yes,...,No,No,Month-to-month,No,Credit card,101.38,7299.36,0,1,0
4,CUST0004,Male,1,No,No,25,Yes,Yes,DSL,No,...,No,Yes,Month-to-month,No,Electronic check,52.22,1305.50,1,1,1


In [984]:
# Comparar las predicciones con los valores reales
correct_prediction = (df3['Churn_pred_1'] == df3['Churn']).sum()

In [985]:
total_predicciones = len(df3)
precisión = (correct_prediction / total_predicciones) * 100
print(f'Precisión: {precisión:.2f}%')

margen_de_error = 100 - precisión
print(f'Margen de error: {margen_de_error:.2f}%')

Precisión: 49.10%
Margen de error: 50.90%


In [986]:
df3['Churn'].value_counts()

Churn
0    2982
1    2898
Name: count, dtype: int64

In [987]:
df3['Churn_pred_1'].value_counts()

Churn_pred_1
0    3263
1    2617
Name: count, dtype: int64

In [988]:
df3['Churn_pred'].value_counts()

Churn_pred
1    4296
0    1584
Name: count, dtype: int64

# 3. Evaluación 

```python

oculta  = tf.keras.layers.Dense(units=128,activation='relu',input_shape=[40])
dropout = tf.keras.layers.Dropout(0.3)
oculta1 = tf.keras.layers.Dense(units=64,activation='relu')
dropout1 = tf.keras.layers.Dropout(0.3)
oculta2 = tf.keras.layers.Dense(units=32,activation='relu')
dropout2 = tf.keras.layers.Dropout(0.3)
oculta3 = tf.keras.layers.Dense(units=16,activation='relu')
dropout3 = tf.keras.layers.Dropout(0.3)
oculta4 = tf.keras.layers.Dense(units=8,activation='relu')
dropout4 = tf.keras.layers.Dropout(0.3)
salida = tf.keras.layers.Dense(units=1, activation='sigmoid')

1000 epochs

40 capas de entrada

5 capas ocultas con [128,64,32,16,8] neuronas cada una y la función de activación ReLU. ReLU convierte los valores negativos a 0 y deja pasar los valores positivos, ayudando a que las redes neuronales aprendan patrones de mejor manera.

1 dropout es una tecnica que permite regularizar que previene el sobreajuste. Durante el entrenamiento que,
desactiva un porcentaje de las neuronas (0.3 [En este caso, el 30%]), lo que fuerza ala red de no depender demasiado
de ciertas neuronas y aprender caracteristicas más robustas

1 capa de salida con 1 neurona de salida con la funcion 'sigmoid',
la cual interpreta la salida de la ultima capa con un valor entre 0 y 1

historial = modelo.fit(X_train,y_train,epochs=1000,verbose=False,validation_split=0.2,callbacks=[early_stopping]) / Se le agrego el validation_split,
el cual utiliza el 20% de mis datos para la validación y el 80% se utilizaran para entrenar el modelo

**early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)** / El early_stopping

basicamente su función es que al momento de que mi modelo este entrenando si ve que ya no esta aprendiendo, se detiene, en este caso lo configure para 10 epocas, y esto ayuda a evitar el sobreajuste, además captura la mejor versión del modelo antes de detenerse

Precisión: 49.29%
Margen de error: 50.71%

Ya captura el valor entre 0 y 1


# Peso

La fórmula para calcular los pesos de las clases en un problema de clasificación binaria es:

$$
\text{peso}_i = \frac{N}{2 \times N_i}
$$

Donde:
- \( N \) es el número total de muestras.
- \( N_i \) es el número de muestras de la clase \( i \).

### Ejemplo

Supongamos que tienes un conjunto de datos con la siguiente distribución de clases:

- Clase 0: 4091 muestras
- Clase 1: 1789 muestras
- Total de muestras: 5880

Cálculo para la Clase 0:

$$
\text{peso}_0 = \frac{5880}{2 \times 4091} \approx 0.72
$$

Cálculo para la Clase 1:

$$
\text{peso}_1 = \frac{5880}{2 \times 1789} \approx 1.64
$$

### Interpretación

- El peso de la clase "0" (mayoritaria) es aproximadamente 0.72, lo que indica que durante el entrenamiento, las muestras de la clase "0" tendrán menos influencia en la actualización de los parámetros del modelo.
- El peso de la clase "1" (minoritaria) es aproximadamente 1.64, lo que indica que durante el entrenamiento, las muestras de la clase "1" tendrán más influencia en la actualización de los parámetros del modelo.